In [1]:
import cv2
import numpy as np
from keras.models import load_model
import mediapipe as mp
from twilio.rest import Client

account_sid = '*******************************'
auth_token = '********************************'

svm = load_model(r"C:\Users\Kanika\OneDrive\Desktop\SignDialer\American Sign Language Digits Dataset\ann.h5")


mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                       min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def image_processed(hand_img):
   
    img_rgb = cv2.cvtColor(hand_img, cv2.COLOR_BGR2RGB)  
    img_flip = cv2.flip(img_rgb, 1) 
    
    
    results = hands.process(img_flip)
    
    
    if results.multi_hand_landmarks:
        
        hand_landmarks = results.multi_hand_landmarks[0]
        landmarks = np.array([[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark]).flatten()
        return landmarks
    else:
        
        return np.zeros(63)

def preprocess_image(hand_img):
    
    img_rgb = cv2.cvtColor(hand_img, cv2.COLOR_BGR2RGB) 
    img_flip = cv2.flip(img_rgb, 1) 
    return img_flip

def predict_sign_language(image):
    data = image_processed(image)  
    data = np.array(data)  
    data = np.expand_dims(data, axis=0)
    y_pred = svm.predict([data])   
    return y_pred[0]               

def make_a_call(predicted_signs):
    str_list = ["+91"]+ [str(num) for num in predicted_signs ]
    result_str1 = ''.join(str_list)
    client = Client(account_sid,auth_token)

    call = client.calls.create(
        twiml = '<Response><Say>hello success learners</Say></Response>',
        to = result_str1,
        from_ = '+12513222571'
)


    



def main():
    predicted_signs =[]
    current_sign=''
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    while True:
        ret, frame = cap.read()
        cv2.rectangle(frame, (0, 40), (300, 400), (255, 255, 255), 2)
        cv2.imshow("Webcam", frame)
        
        
        roi = frame[40:400, 0:300]
       
        
        y_pred = predict_sign_language(roi)
        ans = np.argmax(y_pred)
        
        cv2.putText(roi, str(ans), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        key = cv2.waitKey(1)
        if key == 13:
            predicted_signs.append(ans)
            print("sign saved",ans)
        elif key == ord('q'):
            break
        
        cv2.imshow("roi", roi)
        
        
    print(predicted_signs)
    make_a_call(predicted_signs)
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


1/1 [==============================] - 0s 34ms/step
[]


TwilioRestException: HTTP 400 error: Unable to create record: The phone number you are attempting to call, +91, is not valid.

: 